# StyleGAN2-ada operations

### Preparation


**Run this cell after each session restart**

In [ ]:
#@title General setup { display-mode: "form", run: "auto" }

!pip install torch==1.7.1 torchvision==0.8.2

from IPython.display import HTML, Image, display
from moviepy.editor import ImageSequenceClip, ipython_display
import ipywidgets as widgets
import os
import numpy as np

!apt-get -qq install ffmpeg
!pip install ninja
from google.colab import drive
drive.mount('/G', force_remount=True)
gdir = !ls /G/
gdir = '/G/%s/' % str(gdir[0])
%cd $gdir

#@markdown Copying StyleGAN2 to the directory below on your Google drive (creating it, if it doesn't exist):
work_dir = 'sg2a_eps' #@param {type:"string"}
#@markdown NB: All paths below are relative to this directory (except the archive with source images on the next step). 

#@markdown NB: Avoid connecting Google drive manually via the icon in Files section on the left. Doing so may break further operations.

work_dir = gdir + work_dir + '/'
if not os.path.isdir(work_dir):
  !git clone git://github.com/eps696/stylegan2ada $work_dir
%cd $work_dir
!pip install -r requirements.txt

from src.util.utilgan import file_list, img_list, basename
model = model_pkl = ''
def model_select(work_dir):
  models = file_list(work_dir, 'pkl', subdir=True)
  global model, model_pkl
  model_pkl = models[0]
  model = model_pkl.replace('.pkl', '')
  models = [m.replace(work_dir, '') for m in models if not basename(m) in ['submit_config', 'vgg16_zhang_perceptual']]
  def on_change(change):
    global model, model_pkl
    if change['type'] == 'change' and change['name'] == 'value':
      model = change['new']
      model = os.path.splitext(model)[0] # filename without extension => load custom network
      model_pkl = model + '.pkl' # filename with extension => load original network
  model_select = widgets.Dropdown(options=models, description='Found models:', style={'description_width': 'initial'}, layout={'width': 'max-content'})
  display(model_select)
  model_select.observe(on_change)
# model_select(work_dir)

# Hardware check
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil 
import GPUtil as GPU
gpu = GPU.getGPUs()[0]
!nvidia-smi -L
print("GPU RAM {0:.0f}MB | Free {1:.0f}MB)".format(gpu.memoryTotal, gpu.memoryFree))
print('\nDone!')

## Training

First, let's prepare the dataset. Ensure all your images have the same color channels (monochrome, RGB or RGBA). If you edit the images yourself (e.g. for non-square aspect ratios, or to keep the compositions), ensure their correct size.  
For conditional model the images should be split by subfolders (mydata/1, mydata/2, ..).

Upload zip-archive with images onto Google drive and type its path here (relative to G-drive root). 

If you work with patterns or shapes (rather than compostions), you may want to enable `multicrop` to crop square fragments from bigger images, effectively increasing amount of data (not suitable for conditional data, as it would break folder structure!). The images will be cut into `size`px fragments, overlapped with `step` shift by X and Y. If the image is smaller, it will be upscaled to the `size`. Edit these values according to your dataset.   
*Restart session (Runtime) after `multicrop` run*. 

In [ ]:
#@title Data setup 
dataset = 'test' #@param {type:"string"}
source = 'source.zip' #@param {type:"string"}
content_data = os.path.join('/content', dataset)
data_dir = os.path.join(work_dir, 'data/')
dataset_dir = os.path.join(data_dir, dataset)

#@markdown Multicrop
multicrop = False  #@param {type:"boolean"}
size = "256" #@param [256,512,1024]
overlap = 128 #@param {type:"integer"}
size = int(size)

# cleanup previous attempts
!rm -rf /content/tmp 
!rm -rf $content_data
!rm -rf $dataset_dir

!mkdir /content/tmp
%cd /content/tmp
fpath = os.path.join(gdir, source)
!unzip -o -q $fpath -d $dataset
%cd $work_dir

if multicrop:
  if os.path.isdir('/content/tmp'):
    %run src/util/multicrop.py --in_dir /content/tmp --out_dir $content_data --size $size --step $overlap
    !mv $content_data $data_dir
  else:
    print('/content/tmp not found!!')
else:
  !mv /content/tmp/* $data_dir


Now, we can train StyleGAN2 on the prepared dataset.

In [ ]:
#@title Train
%cd $work_dir
dataset = 'test' #@param {type:"string"}
kimg = 5000 #@param {type:"integer"}
data_dir = os.path.join(work_dir, 'data', dataset)

%run src/train.py --data $data_dir --kimg $kimg

> This will run training process, according to the options and settings in `src/train.py` (check and explore those!!). Results (models and samples) are saved under `train` directory, similar to original Nvidia approach. There are two types of models saved: compact (containing only Gs network for inference) as `<dataset>-...pkl` (e.g. `test-256-0360.pkl`), and full (containing G/D/Gs networks for further training) as `snapshot-...pkl`. Check sample images there to follow the progress.

> The length of the training is defined by `--kimg X` argument (training duration in thousands of images). Reasonable `kimg` value for full training from scratch is 5000-8000, while for finetuning in `--d_aug` mode 1000-2000 may be sufficient.  

> *NB: If you get `RuntimeError: context has already been set` (e.g. after training interruption or `multicrop` run) - restart session, run General setup and Train again.*

Other training options:

In [ ]:
%run src/train.py --help

If the training process was interrupted, resume it from the last saved model.

In [ ]:
#@title Resume training
%cd $work_dir
dataset = 'test' #@param {type:"string"}
resume_dir = 'train/000-test-256-auto-gf_bnc' #@param {type:"string"}
kimg = 5000 #@param {type:"integer"}
data_dir = os.path.join(work_dir, 'data', dataset)

%run src/train.py --data $data_dir --resume $resume_dir --kimg $kimg 

NB: In most cases it's much easier to use a "transfer learning" trick, rather than perform full training from the scratch. For that, we use existing well-trained model as a starter, and "finetune" (uptrain) it with our data. This works pretty well, even if our dataset is very different from the original model. 

So here is a faster way to train our GAN (presuming we have full trained model `train/ffhq-256.pkl` already):

In [ ]:
#@title Finetune training
%cd $work_dir
dataset = 'test' #@param {type:"string"}
resume_pkl = 'train/ffhq-512.pkl' #@param {type:"string"}
kimg = 1000 #@param {type:"integer"}
data_dir = os.path.join(work_dir, 'data', dataset)

%run src/train.py --data $data_dir --resume $resume_pkl --kimg $kimg 

## Generation

Let's produce some imagery from the original cat model (get it [here](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-cat-config-f.pkl) and put onto Google drive within our working directory).  
Generated results are saved as sequences and videos (by default, under `_out` directory).  
More cool models can be found [here](https://github.com/justinpinkney/awesome-pretrained-stylegan2).

In [ ]:
#@title ### Generator setup
!cd $work_dir

output = '_out/cats' #@param {type:"string"}

frames = 50 #@param {type:"integer"}
frame_step = 10 #@param {type:"integer"}
timeframe = '%d-%d' % (frames, frame_step)

cubic_smooth = True #@param {type:"boolean"}
gauss_smooth = False #@param {type:"boolean"}
cubic_smooth = '--cubic ' if cubic_smooth else ''
gauss_smooth = '--gauss ' if gauss_smooth else ''
smooth = cubic_smooth + gauss_smooth

save_lat = False #@param {type:"boolean"}
save_lat = '--save_lat ' if save_lat else ''

seed = 0 #@param {type:"integer"}

#@markdown Select model from the dropdown below:
model_select(work_dir)

> This means loading the model, and producing 50 frames, interpolating between random latent (`z`) space keypoints, with a step of 10 frames between keypoints. 
`save_lat` option = save all traversed dlatent (`w`) points as Numpy array in `*.npy` file (useful for further curating). `cubic` option changes linear interpolation to cubic for smoother animation; `gauss` provides additional smoothing. Set `seed` value to produce repeatable results (0 = random).

In [ ]:
#@title ### Native generation
%cd $work_dir
%run src/_genSGAN2.py --model $model_pkl --out_dir $output --frames $timeframe $smooth $save_lat --seed $seed
ipython_display(ImageSequenceClip(img_list(output), fps=25), center=False)

In [ ]:
#@title ### Custom size generation

sizeX = 400 #@param {type:"integer"} 
sizeY = 300 #@param {type:"integer"}
size = '%d-%d' % (sizeX, sizeY)
scaling = 'pad' #@param ['pad', 'padside', 'symm', 'symmside']

%cd $work_dir
%run src/_genSGAN2.py --model $model --out_dir $output --frames $timeframe --size $size --scale_type $scaling  $smooth $save_lat --seed $seed
ipython_display(ImageSequenceClip(img_list(output), fps=25), center=False)

In [ ]:
#@title ### Multi-latent  generation

sizeX = 768 #@param {type:"integer"} 
sizeY = 256 #@param {type:"integer"}
size = '%d-%d' % (sizeX, sizeY)
scaling = 'pad' #@param ['pad', 'padside', 'symm', 'symmside']

split_X =  3#@param {type:"integer"} 
split_Y =  1#@param {type:"integer"}
split = '%d-%d' % (split_X, split_Y)
splitfine = 0. #@param {type:"number"}

%cd $work_dir
%run src/_genSGAN2.py --model $model --out_dir $output --frames $timeframe --size $size --scale_type $scaling -n $split --splitfine $splitfine $smooth --seed $seed
ipython_display(ImageSequenceClip(img_list(output), fps=25), center=False)

> Here we get animated composition of 3 independent frames, blended together horizontally.  
`splitfine` controls boundary fineness (0 = smoothest/default, higher => thinner).  

Instead of frame splitting, we can load external mask from b/w image file (or folder with image sequence):

In [ ]:
#@title ### Masked  generation

sizeX = 400 #@param {type:"integer"} 
sizeY = 300 #@param {type:"integer"}
size = '%d-%d' % (sizeX, sizeY)

lat_mask = '_in/mask.jpg' #@param {type:"string"} 

%cd $work_dir
%run src/_genSGAN2.py --model $model --out_dir $output --frames $timeframe --size $size --latmask $lat_mask  $smooth --seed $seed
ipython_display(ImageSequenceClip(img_list(output), fps=25), center=False)

`digress` adds some funky displacements by tweaking initial constant layer.  
`truncation` controls variety, as usual  (0 = boring, >1 = weird). 

In [ ]:
#@title ### Other tricks

sizeX = 400 #@param {type:"integer"} 
sizeY = 300 #@param {type:"integer"}
size = '%d-%d' % (sizeX, sizeY)

digress =  2#@param {type:"number"} 
truncation =  1.5#@param {type:"number"} 

%cd $work_dir
%run src/_genSGAN2.py --model $model --out_dir $output --frames $timeframe --size $size --digress $digress --trunc $truncation $smooth --seed $seed
ipython_display(ImageSequenceClip(img_list(output), fps=25), center=False)

## Latent space exploration

For these experiments download [FFHQ model](https://nvlabs-fi-cdn.nvidia.com/stylegan2/networks/stylegan2-ffhq-config-f.pkl) and save to `models` directory.

In [ ]:
#@title ### Generator setup
!cd $work_dir

output = '_out/ffhq' #@param {type:"string"}

frames = 50 #@param {type:"integer"}
frame_step = 10 #@param {type:"integer"}
timeframe = '%d-%d' % (frames, frame_step)

cubic_smooth = True #@param {type:"boolean"}
gauss_smooth = False #@param {type:"boolean"}
cubic_smooth = '--cubic ' if cubic_smooth else ''
gauss_smooth = '--gauss ' if gauss_smooth else ''
smooth = cubic_smooth + gauss_smooth

save_lat = False #@param {type:"boolean"}
save_lat = '--save_lat ' if save_lat else ''

#@markdown Select model from the dropdown list below:
model_select(work_dir)

Project external images onto the model dlatent `w` space, saving points as Numpy arrays in `*.npy` files. 

In [ ]:
#@title ### Images projection

image_dir = '_in/photo' #@param {type:"string"} 
out_dir = '_out/proj' #@param {type:"string"} 
steps =  1000#@param {type:"integer"} 

%cd $work_dir
%run src/projector.py --model $model_pkl --in_dir _in/photo --out_dir _out/proj --steps $steps

from src.util.utilgan import img_list
images = img_list(out_dir, subdir=True)
images = [f for f in images if 'target.jpg' not in f]
Image(images[0], width=512, height=512)

Produce looped animation, interpolating between saved dlatent points (from a file or directory with `*.npy` or `*.npz` files). To select only few frames from a sequence `xxx.npy`, create text file with comma-delimited frame numbers and save it as `xxx.txt` in the same directory (check examples in `_in/dlats`). 

In [ ]:
#@title ### "Walk" between saved dlatent points

dlatents = '_in/dlats' #@param {type:"string"} 
out_dir = '_out/ffhq-dlats' #@param {type:"string"} 

%cd $work_dir
%run src/_play_dlatents.py --model $model --dlatents $dlatents --out_dir $out_dir --fstep $frame_step $smooth
ipython_display(ImageSequenceClip(img_list(out_dir), fps=25), center=False)



We can also load another dlatent point and apply it to higher network layers, effectively "stylizing" images with its' fine features.  
`digress` and `truncation` are also applicable here.

In [ ]:
#@title ### "Walk" between saved points w/tricks

dlatents = '_in/dlats' #@param {type:"string"} 
out_dir = '_out/ffhq-dlats' #@param {type:"string"} 
style_dlat = '_in/blonde458.npy' #@param {type:"string"} 
digress =  2#@param {type:"number"} 
truncation =  1.5#@param {type:"number"} 

%cd $work_dir
%run src/_play_dlatents.py --model $model --dlatents $dlatents --out_dir $out_dir --fstep $frame_step $smooth --style_dlat $style_dlat --digress $digress --trunc $truncation
ipython_display(ImageSequenceClip(img_list(out_dir), fps=25), center=False)

Generate animation by walking from saved dlatent point along feature direction vectors (aging/smiling/etc) one by one.

In [ ]:
#@title ### "Walk" along feature directions

base_lat = '_in/blonde458.npy' #@param {type:"string"} 
vectors = '_in/vectors_ffhq' #@param {type:"string"} 
out_dir = '_out/ffhq_looks' #@param {type:"string"} 

%cd $work_dir
%run src/_play_vectors.py --model $model_pkl --base_lat $base_lat --vector_dir $vectors --out_dir $out_dir --fstep $frame_step
ipython_display(ImageSequenceClip(img_list(out_dir), fps=25), center=False)

> Try discovering more such vectors:
> * https://github.com/genforce/sefa
> * https://github.com/harskish/ganspace